In [1]:
!pip install streamlit PyPDF2 sentence-transformers faiss-cpu cohere


   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ----- ---------------------------------- 2.1/14.6 MB 9.8 MB/s eta 0:00:02
   ----- ---------------------------------- 2.1/14.6 MB 9.8 MB/s eta 0:00:02
   ------------ --------------------------- 4.5/14.6 MB 6.7 MB/s eta 0:00:02
   -------------- ------------------------- 5.2/14.6 MB 6.2 MB/s eta 0:00:02
   ----------------- ---------------------- 6.3/14.6 MB 5.8 MB/s eta 0:00:02
   -------------------- ------------------- 7.6/14.6 MB 5.9 MB/s eta 0:00:02
   ------------------------ --------------- 8.9/14.6 MB 6.0 MB/s eta 0:00:01
   ---------------------------- ----------- 10.5/14.6 MB 6.3 MB/s eta 0:00:01
   ------------------------------ --------- 11.3/14.6 MB 6.0 MB/s eta 0:00:01
   ---------------------------------- ----- 12.6/14.6 MB 5.9 MB/s eta 0:00:01
   -------------------------------------- - 13.9/14.6 MB 6.0 MB/s eta 0:00:01
   ---------------------------------------- 14.6/14.6 MB 6.0 MB/s eta 0:00:00
 

In [ ]:
import os
from PyPDF2 import PdfFileReader
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import cohere

# Initialize models and clients
model = SentenceTransformer('all-MiniLM-L6-v2')
co = cohere.Client('K1AE6XOZ71nkleq8mDhjYD1UPYx7bugskyvvPAGR')

# Function to process PDF and store embeddings
def process_pdf(pdf_path):
    with open(pdf_path, 'rb') as f:
        pdf = PdfFileReader(f)
        document = ''
        for page in range(pdf.getNumPages()):
            document += pdf.getPage(page).extractText()

    # Split the document into segments (e.g., sentences)
    segments = document.split('. ')

    # Generate embeddings for each segment
    embeddings = model.encode(segments)

    # Store embeddings in FAISS index
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    return index, segments

# Function to query the database
def query_database(index, segments, question):
    question_embedding = model.encode([question])
    D, I = index.search(question_embedding, k=5)  # Retrieve top 5 segments
    relevant_segments = [segments[i] for i in I[0]]
    return relevant_segments

# Function to generate a response
def generate_response(question, segments):
    response = co.generate(
        model='xlarge',
        prompt=f"Question: {question}\nContext: {' '.join(segments)}\nAnswer:",
        max_tokens=50
    )
    return response.generations[0].text


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [2]:
# Install necessary libraries
!pip install streamlit PyPDF2 sentence-transformers faiss-cpu cohere

# Create the Streamlit app file using Python code
app_code = """
import os
import streamlit as st
from io import BytesIO
from PyPDF2 import PdfFileReader
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import cohere

# Initialize models and clients
model = SentenceTransformer('all-MiniLM-L6-v2')
co = cohere.Client('K1AE6XOZ71nkleq8mDhjYD1UPYx7bugskyvvPAGR')  # Replace with your actual Cohere API key

# Function to process PDF and store embeddings
def process_pdf(pdf_path):
    with open(pdf_path, 'rb') as f:
        pdf = PdfFileReader(f)
        document = ''
        for page in range(pdf.getNumPages()):
            document += pdf.getPage(page).extractText()

    # Split the document into segments (e.g., sentences)
    segments = document.split('. ')

    # Generate embeddings for each segment
    embeddings = model.encode(segments)

    # Store embeddings in FAISS index
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    return index, segments

# Function to query the database
def query_database(index, segments, question):
    question_embedding = model.encode([question])
    D, I = index.search(question_embedding, k=5)  # Retrieve top 5 segments
    relevant_segments = [segments[i] for i in I[0]]
    return relevant_segments

# Function to generate a response
def generate_response(question, segments):
    response = co.generate(
        model='xlarge',
        prompt=f"Question: {question}\\nContext: {' '.join(segments)}\\nAnswer:",
        max_tokens=50
    )
    return response.generations[0].text

# Streamlit frontend
st.title('QA Bot')
uploaded_file = st.file_uploader('Upload a PDF document', type='pdf')

if uploaded_file is not None:
    # Save the uploaded file to a temporary location
    temp_file_path = '/tmp/uploaded_file.pdf'
    with open(temp_file_path, 'wb') as f:
        f.write(uploaded_file.getbuffer())

    # Process the PDF and store embeddings
    index, segments = process_pdf(temp_file_path)

    # Ask a question
    question = st.text_input('Ask a question')
    if question:
        relevant_segments = query_database(index, segments, question)
        response = generate_response(question, relevant_segments)
        st.write('Answer:', response)
        st.write('Relevant Segments:', relevant_segments)
"""

with open('app.py', 'w') as f:
    f.write(app_code)


In [ ]:
!streamlit run app.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.187.240.132:8501

